In [1]:
# @title MM & Perp Calculator
import ipywidgets as widgets
from IPython.display import display

def calculate_arbitrage_allocation(current_price, ltv, long_liq_price, short_liq_price, liquidation_factor, mmr=0.025):
    # Calculate leverage for long position using liquidation factor
    long_leverage = 1 / (1 - liquidation_factor * (long_liq_price / current_price))

    # Calculate leverage for short position
    # For shorts: using (1 - MMR) as the liquidation factor
    # Similar to longs but using (1 - MMR) instead of liquidation_factor
    short_leverage = 1 / (1 - (1 - mmr) * (current_price / short_liq_price))


    # Calculate capital allocation
    total_leverage = long_leverage + short_leverage
    long_allocation = short_leverage / total_leverage
    short_allocation = long_leverage / total_leverage

    # Calculate capital efficiency
    capital_efficiency = long_allocation * long_leverage

    return {
        'long_allocation': long_allocation,
        'short_allocation': short_allocation,
        'long_leverage': long_leverage,
        'short_leverage': short_leverage,
        'capital_efficiency': capital_efficiency
    }

# Create widgets
current_price_widget = widgets.FloatText(
    value=4.2,
    description='Current Price:',
    style={'description_width': 'initial'}
)

ltv_widget = widgets.FloatSlider(
    value=0.75,
    min=0,
    max=1.0,
    step=0.01,
    description='LTV:',
    readout_format='.4f'
)

liquidation_factor_widget = widgets.FloatSlider(
    value=0.8,  # Default to the same as LTV
    min=0,
    max=1.0,
    step=0.01,
    description='Liquidation Factor:',
    readout_format='.4f',
    style={'description_width': 'initial'}
)

long_liq_price_widget = widgets.FloatText(
    value=3.6,
    description='Long Liq Price:',
    style={'description_width': 'initial'}
)

short_liq_price_widget = widgets.FloatText(
    value=5.2,
    description='Short Liq Price:',
    style={'description_width': 'initial'}
)

# Add widget for total investment
total_investment_widget = widgets.FloatText(
    value=10000,
    description='Total Investment ($):',
    style={'description_width': 'initial'}
)

# Add widget for swap fee
swap_fee_widget = widgets.FloatSlider(
    value=0.003,  # 0.3% default
    min=0,
    max=0.01,
    step=0.0001,
    description='Swap Fee %:',
    readout_format='.4f',
    style={'description_width': 'initial'}
)

# Add widget for MMR
mmr_widget = widgets.FloatSlider(
    value=0.025,  # Default 2.5%
    min=0.001,
    max=0.05,
    step=0.0001,
    description='MMR:',
    readout_format='.4f',
    style={'description_width': 'initial'}
)

output = widgets.Output()

def on_value_change(change):
    with output:
        output.clear_output()
        try:
            result = calculate_arbitrage_allocation(
                current_price_widget.value,
                ltv_widget.value,
                long_liq_price_widget.value,
                short_liq_price_widget.value,
                liquidation_factor_widget.value,
                mmr_widget.value
            )

            # Calculate dollar amounts for each position
            long_amount = total_investment_widget.value * result['long_allocation']
            short_amount = total_investment_widget.value * result['short_allocation']

            # Calculate token quantity and fees for long position
            long_notional = long_amount * result['long_leverage']
            long_token_quantity = long_notional / current_price_widget.value
            long_swap_fee = long_notional * swap_fee_widget.value

            # Calculate fees for short position (0.01% + $1)
            short_notional = short_amount * result['short_leverage']
            short_fee = (short_notional * 0.0001) + 1

            print("\nResults:")
            print(f"Long position: Allocate {result['long_allocation']*100:.2f}% of capital (${long_amount:,.2f}) with {result['long_leverage']:.2f}x leverage")
            print(f"Long Position Token Quantity: {long_token_quantity:.4f} tokens")
            print(f"Long Position Swap Fee: ${long_swap_fee:.2f}")
            print(f"\nShort position: Allocate {result['short_allocation']*100:.2f}% of capital (${short_amount:,.2f}) with {result['short_leverage']:.2f}x leverage")
            print(f"Short Position Fee: ${short_fee:.2f}")
            print(f"\nTotal Transaction Fees: ${(long_swap_fee + short_fee):.2f}")
            print(f"Capital Efficiency: {result['capital_efficiency']:.2f}x")

            # Calculate and display notional exposure
            print(f"\nNotional Exposure:")
            print(f"Long Exposure: ${long_notional:,.2f}")
            print(f"Short Exposure: ${short_notional:,.2f}")

        except Exception as e:
            print(f"Error: {str(e)}")

# Observe changes in ALL widgets
current_price_widget.observe(on_value_change, names='value')
ltv_widget.observe(on_value_change, names='value')
liquidation_factor_widget.observe(on_value_change, names='value')
long_liq_price_widget.observe(on_value_change, names='value')
short_liq_price_widget.observe(on_value_change, names='value')
total_investment_widget.observe(on_value_change, names='value')
swap_fee_widget.observe(on_value_change, names='value')
mmr_widget.observe(on_value_change, names='value')

# Trigger initial calculation
on_value_change({'new': None})

# Display all widgets
display(
    current_price_widget,
    ltv_widget,
    liquidation_factor_widget,
    long_liq_price_widget,
    short_liq_price_widget,
    total_investment_widget,
    swap_fee_widget,
    mmr_widget,
    output
)

FloatText(value=4.2, description='Current Price:', style=DescriptionStyle(description_width='initial'))

FloatSlider(value=0.75, description='LTV:', max=1.0, readout_format='.4f', step=0.01)

FloatSlider(value=0.8, description='Liquidation Factor:', max=1.0, readout_format='.4f', step=0.01, style=Slid…

FloatText(value=3.6, description='Long Liq Price:', style=DescriptionStyle(description_width='initial'))

FloatText(value=5.2, description='Short Liq Price:', style=DescriptionStyle(description_width='initial'))

FloatText(value=10000.0, description='Total Investment ($):', style=DescriptionStyle(description_width='initia…

FloatSlider(value=0.003, description='Swap Fee %:', max=0.01, readout_format='.4f', step=0.0001, style=SliderS…

FloatSlider(value=0.025, description='MMR:', max=0.05, min=0.001, readout_format='.4f', step=0.0001, style=Sli…

Output()

In [3]:
import ipywidgets as widgets
from IPython.display import display
import decimal

# Set decimal precision to 18 digits
decimal.getcontext().prec = 18

def calculate_ratio(token_a_amount, token_b_amount):
    """Calculate the ratio between two token amounts"""
    token_a = decimal.Decimal(str(token_a_amount))
    token_b = decimal.Decimal(str(token_b_amount))
    return token_b / token_a if token_a != 0 else decimal.Decimal('0')

def calculate_optimal_split(total_token_a, price_ratio):
    """Calculate optimal split of Token A for LP"""
    total = decimal.Decimal(str(total_token_a))
    ratio = decimal.Decimal(str(price_ratio))
    
    swap_token_a = total / (ratio + 1)
    keep_token_a = total - swap_token_a
    receive_token_b = swap_token_a * ratio
    
    return (keep_token_a, swap_token_a, receive_token_b)

def on_ratio_button_click(b):
    try:
        token_a = decimal.Decimal(str(sample_a_input.value))
        token_b = decimal.Decimal(str(sample_b_input.value))
        ratio = calculate_ratio(token_a, token_b)
        ratio_output.value = f"""Ratio Calculator Results:
Sample Token A: {token_a:,.8f}
Sample Token B: {token_b:,.8f}
Price Ratio (B/A): 1:{ratio:,.8f}"""
        
        # Automatically update the price ratio input for the split calculator
        price_ratio_input.value = float(ratio)
        
    except (ValueError, decimal.InvalidOperation, decimal.DivisionByZero) as e:
        ratio_output.value = f"Error: Please enter valid numbers! ({str(e)})"

def on_split_button_click(b):
    try:
        total_amount = decimal.Decimal(str(total_amount_input.value))
        price_ratio = decimal.Decimal(str(price_ratio_input.value))
        
        keep_a, swap_a, receive_b = calculate_optimal_split(total_amount, price_ratio)
        
        split_output.value = f"""LP Split Calculator Results:

Your Total Token A: {total_amount:,.8f}
Current Price Ratio (B/A): 1:{price_ratio:,.8f}

Recommended Actions:
1. Keep Token A: {keep_a:,.8f}
2. Swap Token A: {swap_a:,.8f}
3. You'll Receive Token B: {receive_b:,.8f}

Final LP Position:
- Token A: {keep_a:,.8f}
- Token B: {receive_b:,.8f}"""
        
    except (ValueError, decimal.InvalidOperation, decimal.DivisionByZero) as e:
        split_output.value = f"Error: Please enter valid numbers! ({str(e)})"

# Create widgets for ratio calculator
sample_a_input = widgets.FloatText(
    value=1.0,
    description='Sample Token A:',
    style={'description_width': 'initial'}
)

sample_b_input = widgets.FloatText(
    value=1.0,
    description='Sample Token B:',
    style={'description_width': 'initial'}
)

ratio_button = widgets.Button(
    description='Calculate Ratio',
    button_style='info'
)

ratio_output = widgets.Textarea(
    value='Ratio results will appear here...',
    layout=widgets.Layout(width='50%', height='100px')
)

# Create widgets for split calculator
total_amount_input = widgets.FloatText(
    value=100.0,
    description='Total Token A Available:',
    style={'description_width': 'initial'}
)

price_ratio_input = widgets.FloatText(
    value=1.0,
    description='Price Ratio (B/A):',
    style={'description_width': 'initial'}
)

split_button = widgets.Button(
    description='Calculate Split',
    button_style='primary'
)

split_output = widgets.Textarea(
    value='Split results will appear here...',
    layout=widgets.Layout(width='50%', height='200px')
)

# Connect button clicks to handlers
ratio_button.on_click(on_ratio_button_click)
split_button.on_click(on_split_button_click)

# Display widgets
display(widgets.VBox([
    widgets.HTML(value="<h3>Step 1: Calculate Price Ratio</h3>"),
    widgets.HTML(value="<p>Enter sample amounts to calculate the ratio:</p>"),
    sample_a_input,
    sample_b_input,
    ratio_button,
    ratio_output,
    widgets.HTML(value="<h3>Step 2: Calculate Optimal Split</h3>"),
    widgets.HTML(value="<p>Enter your total Token A amount to split:</p>"),
    total_amount_input,
    price_ratio_input,
    split_button,
    split_output
]))